In [1]:
!pip install dgl
!pip install torch==1.9.1

     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 831.4 MB 6.3 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.


In [2]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

#define the message and reduce function . u denotes source node .
gcn_msg = fn.copy_u(u='h', out='m')  # copy_u() specifies unary message function . input feature name = u = 'h' , output message name = out = 'm'
gcn_reduce = fn.sum(msg='m', out='h') #the aggregation on a node u only involves summing over the neighbors’ representations

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [3]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        # By entering a local scope, any out-place mutation to the feature 
        # data will not reflect to the original graph, thus making it easier to 
        # use in a function scope (e.g. forward computation of a model).
        with g.local_scope():
            g.ndata['h'] = feature    # set nodedata
            g.update_all(gcn_msg, gcn_reduce)   # update the graph using message function and reduce function 
            h = g.ndata['h']      
            return self.linear(h)       # pass h through linear layer

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16) #gcn layer 1
        self.layer2 = GCNLayer(16, 7) #gcn layer 2

    # forward pass
    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))      #relu activation in hidden layer 
        x = self.layer2(g, x)
        return x
net = Net()
print(net)    # print the network

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [8]:
from dgl.data import CoraGraphDataset    

# load cora dataset 
def load_cora_data():
    dataset = CoraGraphDataset()
    g = dataset[0]  
    print(g)                # only one graph is present in cora dataset 
    features = g.ndata['feat']      # graph node features 
    labels = g.ndata['label']       # node labels 
    train_mask = g.ndata['train_mask']    #
    test_mask = g.ndata['test_mask']
    print( test_mask)
    return g, features, labels, train_mask, test_mask

load_cora_data()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})
tensor([False, False, False,  ...,  True,  True,  True])


(Graph(num_nodes=2708, num_edges=10556,
       ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
       edata_schemes={}), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([3, 4, 4,  ..., 3, 3, 3]), tensor([ True,  True,  True,  ..., False, False, False]), tensor([False, False, False,  ...,  True,  True,  True]))

In [11]:
def evaluate(model, g, features, labels, mask):
    model.eval()                            # model evaluation 
    with th.no_grad():                      # disabling gradient calculation as it is not required in  test phase
        logits = model(g, features)         # get logits
        logits = logits[mask]               # apply mask
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)   # get percentage of accuracy 


# evaluate(net, g, features, labels, test_mask)

In [12]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)        # adam optimiser
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()                                             # train network
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask]) # calculate loss using negetive log likelihood

    optimizer.zero_grad()                                   # Sets the gradients of all optimized tensors to zero.
    loss.backward()                                         # back propagation 
    optimizer.step()                                        #

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)      # calculate accuracy
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(epoch, loss.item(), acc, np.mean(dur)))

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})
tensor([False, False, False,  ...,  True,  True,  True])
logits tensor([[ 0.0804, -0.1010, -0.1894,  ...,  0.2333, -0.1182, -0.1701],
        [ 0.1302, -0.0741, -0.2311,  ...,  0.2990, -0.1073, -0.1473],
        [-0.1045, -0.2556,  0.1757,  ...,  0.7600, -1.0007, -0.4314],
        ...,
        [ 0.1418, -0.0594, -0.2369,  ...,  0.1466,  0.0591, -0.1324],
        [ 0.0648, -0.1275, -0.1705,  ...,  0.2288, -0.1830, -0.2041],
        [-0.0351, -0.2037,  0.0332,  ...,  0.3469, -0.5150, -0